In [ ]:
# Global imports
import warnings

warnings.filterwarnings("ignore")

import re
import numpy as np
import pandas as pd
import seaborn as sn
# Please install langdetect
from langdetect import detect

from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
food_facts_data = pd.pandas.read_csv('data/en.openfoodfacts.org.products.csv', sep='\t', parse_dates=True, low_memory=False)

In [ ]:
# There are a lot of NaN values in the dataset
food_facts_data.isnull().mean().sort_values(ascending=True).plot.barh(figsize = (8, 30))

### First idea
We are very curious in "Food carbon footprint": greenhouse gas emissions produced by growing, rearing, farming, processing, transporting, storing, cooking and disposing of the food we eat. We are interested in finding out the countries, which are emitting huge amount of carbon in the result of food production and consumption, and identify the ways to reduce it. We hope the dataset will give us insights about the products and their origins having highest carbon footprint(i.e. meat, cheese,eggs, so on) and the ones having lower carbon footprints(i.e. fruit, vegetables, beans, nuts so on). Additionally we consider that food packaging and food waste treatment have huge proportion of impact on carbon emission, thus these were hypothesis that we were interested to test and make conclusions about global problems.

In [ ]:
CARBON_FOOD_PRINT_MASK = ~food_facts_data['carbon-footprint_100g'].isnull()
print("Number of food samples with valid carbon foot print is {}".format(food_facts_data[CARBON_FOOD_PRINT_MASK].shape[0]))

In [ ]:
# Distribution of Carbon-foot-print.
food_facts_data['carbon-footprint_100g'].hist(bins=50)

In [ ]:
numeric_columns = food_facts_data.dtypes[food_facts_data.dtypes ==  'float64'].index

# Plotting the correlation matrix for carbon food print
corr_ = food_facts_data[CARBON_FOOD_PRINT_MASK][numeric_columns].corr().abs()
fig, axis = plt.subplots(figsize=(10, 8))
sn.heatmap(corr_, mask=np.zeros_like(corr_, dtype=np.bool), cmap=sn.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=axis)

In [ ]:
## Sort the features having highest correlation with carbon footprint
valid_corr_columns = corr_['carbon-footprint_100g'].dropna().sort_values(ascending=False)
valid_corr_columns = valid_corr_columns[valid_corr_columns > 0.4]

In [ ]:
# Checking out number of data samples with high-correlation with other non-numeric columns.
food_facts_data[food_facts_data['carbon-footprint_100g'].notnull()][valid_corr_columns.index].notnull().sum()

## Conclusion

Unfortunately the high correlation is because of the lack of datapoints, it has been shown that the proportions of NAN is very high in the dataset where the carbon footprint has some information. So our thoughts that we may apply regression and other fill in tools to fill the carbon footprint values are ruined, we decided to leave this idea because we dont have enough data and didn't find the carbon footprint of each food product to make real conculsions. 

### Switch to the second idea
 Our final idea is  "Food allergy", which happens when human body overreact to exposure to particular substances in the food.
Our aim in this idea is to identify the possible allergies that unlabeled food may cause and recommend the foods that has specific nutrition  and aren't affecting to people with specific allergies.
The social goodness will be to help food allergic people find their healthy and enough proportion of  food easily.


So we will mostly observe the Additives, Ingredients, Nutrients, Allergens and preprocess them for our next milestone.

## Additives and Nutrients analysis

In [ ]:
# Plotting distribution of Additives in the dataset
number_additives = food_facts_data[food_facts_data.additives_en.notnull()].additives_en.str.split(',').values.flatten()
number_additives = np.concatenate(number_additives)
number_additives = np.unique(number_additives, return_counts=True)
number_additives = pd.Series(number_additives[1], index=number_additives[0])

In [ ]:
number_additives.sort_values()[-20:].plot.barh()
plt.title("The top additives in the dataset")

In [ ]:
print("the number of unique additives in the allergens dataset is {}".format(number_additives.shape[0]))

According to the documentation of dataset each food item is assigned a unique code. So we can make code as the index of our data frame and relate all the other datasets on this index.

However, we find that the code is not unique.

In [ ]:
try:
    food_facts_data.set_index('code', inplace=True, drop=False)
except:
    print('code is already index')
    
print("Is the dataset index unique? {}".format(food_facts_data.index.is_unique))

In [ ]:
# Lets investigate the data samples with the same bar code.
_not_unique = food_facts_data[['code']].groupby(food_facts_data.index).count() != 1
food_facts_data.loc[_not_unique[_not_unique.code].index].head()

We see that the data contains duplicate values for all columns, except for last_modified time. Since the food product is same we pick the latest samples.

In [ ]:
food_facts_data.drop(columns=['code'], inplace=True)
food_facts_data = food_facts_data.groupby(food_facts_data.index).head(1)

In [ ]:
print("Is index unique: {}".format(food_facts_data.index.is_unique))
print("Number of na index: {}".format(food_facts_data.index.isna().sum()))

In [ ]:
# Remove the na code food item as the data is corrupted
food_facts_data = food_facts_data.drop(np.nan)

In [ ]:
print("Number of na index: {}".format(food_facts_data.index.isna().sum()))

We first analyse the nutritional facts of all the available products. So we keep columns 'product_name', 'categories'
and all nutrition features (number of features: 85)

In [ ]:
NUTRIENTS_ALL = [
       'energy_100g', 'energy-from-fat_100g', 'fat_100g',
       'saturated-fat_100g', '-butyric-acid_100g', '-caproic-acid_100g',
       '-caprylic-acid_100g', '-capric-acid_100g', '-lauric-acid_100g',
       '-myristic-acid_100g', '-palmitic-acid_100g', '-stearic-acid_100g',
       '-arachidic-acid_100g', '-behenic-acid_100g',
       '-lignoceric-acid_100g', '-cerotic-acid_100g',
       '-montanic-acid_100g', '-melissic-acid_100g',
       'monounsaturated-fat_100g', 'polyunsaturated-fat_100g',
       'omega-3-fat_100g', '-alpha-linolenic-acid_100g',
       '-eicosapentaenoic-acid_100g', '-docosahexaenoic-acid_100g',
       'omega-6-fat_100g', '-linoleic-acid_100g',
       '-arachidonic-acid_100g', '-gamma-linolenic-acid_100g',
       '-dihomo-gamma-linolenic-acid_100g', 'omega-9-fat_100g',
       '-oleic-acid_100g', '-elaidic-acid_100g', '-gondoic-acid_100g',
       '-mead-acid_100g', '-erucic-acid_100g', '-nervonic-acid_100g',
       'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g',
       'sugars_100g', '-sucrose_100g', '-glucose_100g', '-fructose_100g',
       '-lactose_100g', '-maltose_100g', '-maltodextrins_100g',
       'starch_100g', 'polyols_100g', 'fiber_100g', 'proteins_100g',
       'casein_100g', 'serum-proteins_100g', 'nucleotides_100g',
       'salt_100g', 'sodium_100g', 'alcohol_100g', 'vitamin-a_100g',
       'beta-carotene_100g', 'vitamin-d_100g', 'vitamin-e_100g',
       'vitamin-k_100g', 'vitamin-c_100g', 'vitamin-b1_100g',
       'vitamin-b2_100g', 'vitamin-pp_100g', 'vitamin-b6_100g',
       'vitamin-b9_100g', 'folates_100g', 'vitamin-b12_100g',
       'biotin_100g', 'pantothenic-acid_100g', 'silica_100g',
       'bicarbonate_100g', 'potassium_100g', 'chloride_100g',
       'calcium_100g', 'phosphorus_100g', 'iron_100g', 'magnesium_100g',
       'zinc_100g', 'copper_100g', 'manganese_100g', 'fluoride_100g',
       'selenium_100g', 'chromium_100g', 'molybdenum_100g', 'iodine_100g',
       'caffeine_100g', 'taurine_100g', 'ph_100g']

In [ ]:
nutrition_facts = food_facts_data[NUTRIENTS_ALL]

In [ ]:
nutrition_facts.describe()

We can observe that some nutrients have only very few data points. (For example -butyric-acid has only one data point)
So, we keep only top KEEP_NUTRITION nutrients at the moment so that count is greater than 1000.

In [ ]:
count_nutrition = nutrition_facts.count().sort_values(ascending=False)
count_nutrition = count_nutrition[count_nutrition>1000.]
count_nutrition.plot.barh(figsize=(16,10))

In [ ]:
NUTRIENTS_FEATURES = count_nutrition.index.values.tolist()
# Now we have 36 nutrients columns.
# 'energy_100g', 'proteins_100g', 'fat_100g', 'carbohydrates_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
# 'saturated-fat_100g', 'fiber_100g', 'cholesterol_100g', 'trans-fat_100g', 'calcium_100g', 'vitamin-c_100g', 'iron_100g',
# 'vitamin-a_100g', 'potassium_100g', 'polyunsaturated-fat_100g', 'monounsaturated-fat_100g', 'vitamin-pp_100g', 
# 'vitamin-b1_100g', 'vitamin-b2_100g', 'alcohol_100g', 'vitamin-d_100g', 'vitamin-b6_100g', 'magnesium_100g', 'phosphorus_100g',
# 'vitamin-b12_100g', 'vitamin-b9_100g', 'zinc_100g', 'folates_100g', 'pantothenic-acid_100g', 'copper_100g', 'vitamin-e_100g',
# 'manganese_100g', 'selenium_100g', 'omega-3-fat_100g'

In [ ]:
nutrition_facts = nutrition_facts[NUTRIENTS_FEATURES]

In [ ]:
nutrition_facts.describe()

We also observe many outliers in the dataset. For example, for some products proteins_100g value is 31000 which should not be
possible as we are measuring the quantities in per 100 g. One possible explaination would be that the units of measurements
are not consistent with other samples. Also, some nutrients values are negative. These values may be as a result of error in
interpreting - sign as negative.
Presently, we remove these outliers from the data.

In [ ]:
def remove_outliers(df):
    # First remove the negative sign
    df = df.abs()
    
    # As the weight of the food item is 100g, the weight of nutrients can never exceed 100g.
    threshold_cutoff = pd.Series(100*np.ones(df.columns.size), index=df.columns)
    
    # Since energy_100g measured in kilo joules(kJ) we don't clip it. However the highest energy is given by fats (37kJ/g).
    # So the upper bound of energy per 100g should be 3700kJ.
    threshold_cutoff.loc['energy_100g'] = 40*1e2
    
    return df[df <= threshold_cutoff].dropna(how='all')

In [ ]:
nutrition_facts_filt = remove_outliers(nutrition_facts)
print("Nutrition data shape {}".format(nutrition_facts_filt.shape))

In [ ]:
nutrition_facts_filt.describe()

In [ ]:
def plot_distributions_nutrition(df, nlog_scale_col):
    fig, axes = plt.subplots(9, len(df.columns)//9, figsize=(30, 30))
    axes = axes.flatten()
    for col, axis in zip(df.columns, axes):
        bins = np.linspace(df[col].min(), df[col].quantile(0.99), 50)
        df.hist(column = col, bins = bins, ax=axis)
        if not col in nlog_scale_col:
            axis.set_yscale('log')

In [ ]:
plot_distributions_nutrition(nutrition_facts_filt, nlog_scale_col=['energy_100g', 'proteins_100g', 'fat_100g',
                                                                  'carbohydrates_100g', 'sugars_100g', 
                                                                   'omega-3-fat_100g'])

If a nutrition value is not mentioned on the label (NaN), we can assume that the nutrient is absent in the food item.
Thus we replace all NaN with 0. This is because in the absence of any nutrition value, we have no way to know if the product
actually has that nutrient. For the outlier products, our proposed solution of recommending alternate food product
based on nutrients (for allergies and additives) can give wrong results. But we see that this problem even arises with OpenFood
website that calcultes nutri-score wrong if outliers are present.



In [ ]:
def remove_nan_values(df):
    df[df.columns.drop('energy_100g')] = df[df.columns.drop('energy_100g')].replace(np.NaN, 0)
    return df

nutrition_facts_filt = remove_nan_values(nutrition_facts_filt)

In [ ]:
print("The shape of nutrition data after missing values removal {}".format(nutrition_facts_filt[nutrition_facts_filt.energy_100g.isna()].shape))

We Observe that there are 6424 products with unknown energy_100g. 
So, for these products we can calculate approximate energy.


$${energy{100g}} = 17*{protein{100g}} + 37*{fat{100g}} + 17*{carbohydrate{100g}} + 8*{fibre{100g}} + 29*{alcohol_{100g}}$$


source: http://www.mydailyintake.net/energy/

In [ ]:
def remove_energy_nan(df):
    na_mask = df['energy_100g'].isna()
    df.loc[df.index[na_mask], 'energy_100g'] = \
                        17*(df[na_mask].proteins_100g + df[na_mask].carbohydrates_100g) +\
                        37*df[na_mask].fat_100g + 8*df[na_mask].fiber_100g + 29*df[na_mask].alcohol_100g
    return df

In [ ]:
nutrition_facts_filt = remove_energy_nan(nutrition_facts_filt)

In [ ]:
food_facts_data.drop(NUTRIENTS_ALL, axis = 1, inplace = True)

In [ ]:
# put all the changes to the original data
food_facts_data = food_facts_data.merge(nutrition_facts_filt, left_index=True, right_index=True)

## Text exploration

Now lets do some research on how ingredients text and allergens are like and what transformation do we need to perform for each one of them

In [ ]:
# Getting all the data where ingredients text and allergens are applicable
food_allergens_mask = food_facts_data["ingredients_text"].notnull() & food_facts_data["allergens"].notnull()
food_allergens = food_facts_data[food_allergens_mask]

In [ ]:
print("The total number of products is: {}".format(food_facts_data.shape[0]))
print("The total number of products with food allergens ando ingredients data is: {}".format(food_allergens.shape[0]))

****This is approximately 10% of the dataset and hence we have more training data then carbon footprint****

In [ ]:
# Get the language for a particular food / allergen product - as to which language is it written in
def detect_language(phrase):
    try:        
        if not phrase:
            return 'No language'
        
        language_id = detect(phrase)
        return language_id
    except:
        pass

In [ ]:
food_allergens['language_id'] = food_allergens["ingredients_text"][:2000].apply(lambda x: detect_language(x))

So we take the first 2000 samples to see how many languages are there
As, we can see there are 22 languages for ingredients_text which is a ***very difficult translation task in its own!***

In [ ]:
food_allergens.groupby('language_id').count()['url'].plot.barh()
plt.title("The frequency of used Languages")
food_allergens = food_allergens.drop(columns=["language_id"])

In [ ]:
# Now trying to find the number of languages present in the allergens column
food_allergens['language_id'] = food_allergens["allergens"][:2000].apply(lambda x: detect_language(x))
food_allergens.groupby('language_id').count()['url'].plot.barh()
plt.title("the number of languages present in the allergens column")
food_allergens = food_allergens.drop(columns=["language_id"])

In [ ]:
allergens_dict = {}
with open("original_allergens.txt") as f, open("english_allergens.txt") as f2:
    for x, y in zip(f, f2):
        allergens_dict[x.lower().replace('\n', '').strip().rstrip()] = re.sub(' +', ' ', y).lower().replace('\n', '').strip().rstrip()

In [ ]:
def split_string_to_list(phrase, match_dict=None):
    # Sometimes the word has a incorrect language tag in front of it
    # So we are removing all tags and using language detection instead
    foods = []

    for food in phrase.split(','):
        food = food.strip().rstrip()
        
        # Delete all additives since additives are nearly defined for the whole dataset
        food = re.sub(' +', ' ', food).lower()
        food = re.sub(r"[^A-Za-zàâçéèêëîïôûùüÿñæœ\'\s,]", '', food)
        
        # or food items which are made up of one characters
        if(len(food) <= 2):
            continue
        
        # Split the tag - en, fr if they have it
        formatted_food = food.split(':')[1] if ':' in food else food
    
        if match_dict and formatted_food in match_dict:
            translated_food = match_dict[formatted_food]
        else:
            translated_food = formatted_food
        foods.append(translated_food)
    return foods

Transferring all french allergens or allergens in any other language to the english language from the translated files we have

In [ ]:
food_allergens['allergens'] = food_allergens['allergens'].apply((lambda x: split_string_to_list(x, match_dict=allergens_dict)))

In [ ]:
food_allergens['allergens'] = food_allergens['allergens'].apply(frozenset) # Removing all the duplicates from each of the product
unique_allergens = frozenset.union(*food_allergens['allergens'])

print("The total number of food allergens before cleaning them for less than 5 products: {}".format(len(unique_allergens)))

In [ ]:
# Now we get the allergy count for each of the allergies which are present in more than 5 products
allergen_count = {}
temp_allergens = list(unique_allergens)

# Calculating the allergens and removing the allergens which are present in less than 5 products
for allergen in unique_allergens:
    count = food_allergens['allergens'].apply(lambda x: int(allergen in x)).sum()
    
    # Remove allergen if it is present in less than 5 products
    if count < 5:
        temp_allergens.remove(allergen)
        continue
    allergen_count[allergen] = count

In [ ]:
pd.Series(allergen_count).sort_values()[-20:].plot.barh() 
plt.title("top most 20 allergens in the dataset")

In [ ]:
print('Total number of allergens after cleaning them by considering them for greater than 5 products: {}'.format(
    len(allergen_count.keys())))

In [ ]:
# Take the intersection of all allergens which coexist with 
allergen_set = frozenset(allergen_count.keys())
food_allergens['allergens'] = food_allergens['allergens'].apply(lambda x: allergen_set.intersection(x))

In [ ]:
temp_allergens_df = pd.DataFrame(pd.np.empty((food_allergens.shape[0],len(allergen_set))) * 0, columns=allergen_count.keys(), index=food_allergens.index) 

In [ ]:
food_allergens['allergens'] = food_allergens.allergens.apply(list)

In [ ]:
# Making one hot encoding labels for each of the allergen we need to consider in the dataset
for index in food_allergens.index:
    temp_allergens_df.loc[index, food_allergens.loc[index].allergens] = 1

In [ ]:
food_allergens = pd.concat([food_allergens, temp_allergens_df], axis=1)

In [ ]:
food_allergens.head() # we can see that dummy features have been added at the end of the dataset

In [ ]:
# Splitting and flattening the ingredients text to see how many unique ingredients are there in  the total dataset
food_allergens['ingredients_text'] = food_allergens['ingredients_text'].apply((lambda x: split_string_to_list(x)))
food_allergens['ingredients_text'] = food_allergens['ingredients_text'].apply(frozenset)

# Removing all the duplicates from each of the product
unique_ingredients = frozenset.union(*food_allergens['ingredients_text'])

print("The total number of unique ingredients in the dataset are: {}".format(len(unique_ingredients)))

Since, there are 186516 features which are 60 times more than the number of unique allergens (approx 3000). We will need to group these ingredients together under the same umbrella. For this, we are going to go through each of the unique ingredients datasets and get only nouns out from them, and then use word2vec and nltk libraries to group similar ingredients under the higher level ingredient. 

We haven't pursued this task since for this we would need to train a model on all ingredients and this comes under feature engineering and modelling aspect which we will pursue for Milestone 3.
Google translate was used to translate the text feature files, hence the translations are not accurate since they contained several languages, which makes the automatic translation task be difficult.

In [ ]:
# drop the features which are not needed
food_facts_data.drop(['url','image_url', 'image_small_url',
       'image_ingredients_url', 'image_ingredients_small_url',
       'image_nutrition_url', 'image_nutrition_small_url',
        'created_t','last_modified_t', 'generic_name','packaging_tags',
        'manufacturing_places_tags','emb_codes', 'emb_codes_tags','first_packaging_code_geo',
        'cities_tags','countries_tags','no_nutriments', 'additives_tags','cities','ingredients_from_palm_oil',
        'ingredients_that_may_be_from_palm_oil','nutrition_grade_uk', 'nova_group','energy_100g'],
         axis = 1, inplace = True)

In [ ]:
# Save the data as csv file
food_facts_data.to_csv('food_facts.csv')
food_allergens.to_csv('food_allergens.csv')

### We end Milestone 2 here. From here, onwards we follow up with Milestone and the research questions mentioned in the README.

# Feature Engineering - Allergens

In this part of the notebook, we are going to group the allergens into a small categories so that we can be more specific as to how each product has what kind of allergen. The allergens were translated before using Google translate and some manual data cleaning, although it was kind of tedious job since allergens or the text in general is in many languages in the dataset.

For example: milk, milk solids would come under the `milk` related allergen.

In [ ]:
## Putting in the imports for Milestone 3
import os
import re
import nltk
import itertools
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.manifold import TSNE
from collections import Counter
from nltk.stem import PorterStemmer
from gensim.models import KeyedVectors
from nltk.cluster import KMeansClusterer
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
## Loading the file with all the translated allergens
translated_words = []

with open('./english_allergens.txt') as f:
    for line in f.readlines():
        translated_words.append(line)

In [ ]:
print('Total number of english translated allergens: {}'.format(len(translated_words)))

We are converting each of the translated allergens into lowercase for consistency, stripping any allergens which are completely made up of numbers, stripping them of spaces and some of the allergens have question marks and other characters and cleaning them as well.

In [ ]:
# Do a bit of data cleaning on the allergens to make them more consistent
translated_words = [word.lower() for word in translated_words]
translated_words = [re.sub(r'\b[0-9]+\b\s*', '', word).strip().rstrip() for word in translated_words]
translated_words = [word.replace('?', '').replace('-', ' ').replace("'", '') for word in translated_words]

In [ ]:
# Check how many allergens are single words
total_single_words = 0
for word in translated_words:
    if len(word.split(' ')) != 1:
        continue
    else:
        total_single_words += 1
        
print('Total number of allergens which are single word: {}'.format(total_single_words))

Now from above, we know that out of 3168 allergens; about 1517 allergens are single words. So they can be easily fed into the `Word2Vec` model, but we need to take care of the allergens which are **made up of 2 or more words**. 

Below, we try to remove the duplicates since many of the translated allergens end up being translated to the same context.

In [ ]:
single_words = {}
for word in translated_words:
    single_words[word] = word.split(' ')

cnt_single_words = Counter(list(itertools.chain(*single_words.values())))

Now we use the NLTK library for Part of Speech tagging since we are only interested in Nouns and not verbs, adjectives etc. So, we tag each word according to what category it belongs to and remove the ones which are not nouns. Also, consider only words which are made up of characters not empty strings.

In [ ]:
# Using the NLTK library for Part of Speech tagging
for word in single_words:
    if not word:
        continue
    single_words[word] = [w for w in single_words[word] if w]
    single_words[word] = nltk.pos_tag(single_words[word])

In [ ]:
for word in single_words:
    result = []
    if not any(single_words[word]):
        single_words[word] = result
    else:
        for w, tag in single_words[word]:
            if tag.startswith('NN') or tag.startswith('JJ'):
                result.append((w, tag))
            single_words[word] = result

Now we know that some of the allergens are actually additives and these additives generally have a format in which they are written in. For example, `e223`, we remove all of these entries from the dictionary and put them in a list which is labelled for additives.

In [ ]:
all_additives = []

for word in single_words.copy():
    if not bool(re.match(r"e[0-9]{3}", word)):
        continue
    else:
        all_additives.append(word)
        del single_words[word]

We can also remove entries from the dictionary which are of length 1 or 2, since they do not make any sense and hence would be easier for the Word2Vec model to train without noise.

In [ ]:
# remove allergens which are of single length or length 2
for word in single_words.copy():
    if len(word) > 2 and single_words[word]:
        continue
    else:
        del single_words[word]

Now we load the pre existing word2Vec model which has been trained by GLoVE and load it. This has been primarily been trained on Wikipedia dataset and news articles. We will use this model to generate the vector for each of our allergens.

In [ ]:
word2vec_output_file = 'data/word2vec_pretrained.txt'

# Only change the glove file to word2vec file if it does not exist
if not os.path.exists('data/word2vec_pretrained.txt'):
    glove2word2vec('data/glove.840B.300d.txt', word2vec_output_file)

model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
print('Number of vocab unique words: {}'.format(len(model.wv.vocab.items())))

For each of the words which have vectors, we load the vector from the model otherwise we append them to the others list since these are words not present in the model and the `Others` tag can be associated with them.

In [ ]:
## Getting the word embeddings for each of the unique words present
other_words = []

total_words = []
for word in single_words:
    total_words.extend(word.split(' '))
total_words = set(total_words)

word_embedding_matrix = dict()
for word in total_words:
    try:
        word_embedding_matrix[word] = model.wv.__getitem__(word)
    except KeyError:
        other_words.append(word)
        pass

Now all the Word2Vec vectors are in 300 dimensions. To visualize them, we need to reduce them to 2 dimensions and see what clusters are made up of them. For this, we use the algorithm called `TSNE`. This basically clusters based on the distance and taking in account the local and global context of the vectors.

In [ ]:
def tsne_plot(model):
    labels = []
    tokens = []

    for word in word_embedding_matrix:
        tokens.append(word_embedding_matrix[word])
        labels.append(word)
    
    # Perplexity takes into account the global and local features
    # We are using dimensionality reduciton for 2 features and taking 2500 iterations into account
    tsne_model = TSNE(perplexity=40, n_components=2, n_iter=2500, random_state=0)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(20, 20)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(model)

As we can see from above, our Word2Vec works, a really interesting result!.  This can be seen that the chemicals are grouped together on the bottom right, the seafood are grouped together in the upper center of the graph, and we can also see examples that `dehydrated` and `dried` go together, `scotch`, `beer`, `ales` are also grouped together. Hence, our Word2Vec is able to group words of the same context together.

In [ ]:
temp_single_words = single_words.copy()
others_present_in_single = []

for word in temp_single_words:
    for w, t in temp_single_words[word]:
        if not w in other_words:
            continue
        else:
            if len(single_words[word]) == 1:
                del single_words[word]
            else:
                others_present_in_single.append(w)
                single_words[word] = [(i, tag) for i, tag in single_words[word] if w != i]

In [ ]:
other_words = set(other_words) - set(others_present_in_single)

In [ ]:
for word in single_words:
    single_words[word] = [(w, t, word_embedding_matrix[w]) for w, t in single_words[word]]

In [ ]:
allergens_embedding_matrix = dict()

for word in single_words:
    allergens_embedding_matrix[word] = sum([e for w, t, e in single_words[word]])

Now, although we have Word2Vec vectors and they have been clustered correctly on the basis of context. We would like to cluster these words in the allergens together on the basis of `similarity` (cosine similarity metric). For this, we use the K Means clustering algorithm with with cosine as the distance metric. Once, we have assigned them to clusters we assign each of the labels to each word so that we can relate them.

In [ ]:
total_clusters = 5
# Applying Kmeans clusters on vectors  and making 10 clusters out of it
k_means = KMeansClusterer(total_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = k_means.cluster(allergens_embedding_matrix.values(), assign_clusters=True)
print(assigned_clusters[:20])

In [ ]:
assigned_cluster_dict = dict()
for i, word in enumerate(allergens_embedding_matrix.keys()):  
    assigned_cluster_dict[word] =  assigned_clusters[i]
    
# Add additives and others to assigned cluster dict
assigned_cluster_dict.update(dict.fromkeys(all_additives, 5))
assigned_cluster_dict.update(dict.fromkeys(other_words, 6))

In [ ]:
# Change the labels for each cluster and assign labels to it
assigned_labels = pd.DataFrame.from_dict(assigned_cluster_dict, orient='index')
assigned_labels.head()

So, as we have done clustering on each of the allergens and classified them into 7 categories.

0. This contains all kind of `wheat`, `flour`
1. Mostly milk, fruit and nut related products like `butter`, `orange`, `egg`
2. Mostly powdered related products `whey powder` or related to proteins - like `milk protein`, `whey protein`
3. All kinds of acids - like `bisulfite`, `lactate`, `lupine`
4. Mostly sea food related allergens - such as `shellfish`, `capelin`
5. Additives
6. Others - which we have been unable to translate or which not present in the Word2Vec vocab.

In [ ]:
labels = {
    0: 'wheat/flour', 1: 'milk/fruit/nut', 2: 'powder/protein',
    3: 'acids', 4: 'seafood', 5: 'additives', 6: 'others'
}

# Assigning the grouped labels we found each
assigned_labels['grouped'] = assigned_labels[0].apply(lambda x: labels[x])
assigned_labels.head()

In [ ]:
def get_grouped_labels(x):
    result = []
    for i in x:
        print(i, x)
        result.append(assigned_labels[assigned_labels.index == i].values[0][1])
    return result

food_allergens['grouped_allergens'] = food_allergens['allergens'].apply(lambda x: get_grouped_labels(x))